In [3]:
import os
import numpy as np
import pandas as pd
import sys
from tqdm import tqdm
from pydicom import dcmread
import shutil
import subprocess as sp

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 5)

In [4]:
def get_dicom_meta_from_file(dicom_root_dir, patient_id, img_id, extension = 'dcm'):
    dicom_path = os.path.join(dicom_root_dir, f'{patient_id}/{img_id}.{extension}')
    return dcmread(dicom_path, stop_before_pixels=True)


def get_dicom_meta(dicom_root_dir, dicom_df, extension = 'dcm'):
    dicom_data = dict()
    keywords = set()
    dicom_df = dicom_df[["patient_id", "image_id"]].copy()

    for index, row in tqdm(dicom_df.iterrows()):
        patient_id = row["patient_id"]
        image_id = row["image_id"]
        dicom = get_dicom_meta_from_file(dicom_root_dir, patient_id, image_id, extension)
        assert dicom.get('ModalityLUTSequence', None) is None
        if patient_id not in dicom_data:
            dicom_data[patient_id] = dict()
        if image_id not in dicom_data[patient_id]:
            dicom_data[patient_id][image_id] = dict()
        for feature in dicom.iterall():
            dicom_data[patient_id][image_id][feature.keyword] = feature.value
            keywords.add(feature.keyword)
    for keyword in keywords:
        dicom_df[keyword] = dicom_df[["patient_id", "image_id"]].apply(lambda x: np.nan if keyword not in dicom_data[x.patient_id][x.image_id] else dicom_data[x.patient_id][x.image_id][keyword], axis=1)
    print(": Keywords extracted from dicom files:")
    for keyword in keywords:
        print("--> {}".format(keyword))
    return dicom_df

### Competition data 

In [ ]:
comp_df = pd.read_csv('/home/dangnh36/datasets/.comp/rsna/cv/v1/train_fold_0.csv')
print(comp_df.columns)
comp_df

### Vindr

In [ ]:
ROOT_DIR = '/home/dangnh36/datasets/.comp/rsna/external/vindr/'
DCM_ROOT_DIR = os.path.join(ROOT_DIR, 'images')

In [ ]:
df = pd.read_csv(os.path.join(ROOT_DIR, 'breast-level_annotations.csv'))
df

In [ ]:
df.columns

In [ ]:
df.rename(columns = {'study_id': 'patient_id', 'view_position': 'view',
                     'breast_birads': 'BIRADS', 'breast_density': 'density'}, inplace = True)
df.BIRADS = df.BIRADS.apply(lambda x: int(x.split()[-1]))
df.density = df.density.apply(lambda x: x.split()[-1] if type(x) == str else x)
df

In [ ]:
df.density.value_counts()

In [ ]:
df.BIRADS.value_counts()

In [ ]:
meta_df = get_dicom_meta(DCM_ROOT_DIR, df, extension = 'dicom')

In [ ]:
meta_df

In [ ]:
meta_df[~meta_df.VOILUTSequence.isna()]

In [ ]:
arr = np.array(meta_df.at[19965, 'LUTData'])
arr

In [ ]:
arr.max(), arr.mean(), arr.min()

In [ ]:
list(set(arr))

In [ ]:
arr[2048]

In [ ]:
meta_df.to_csv(os.path.join(ROOT_DIR, 'dicom_meta_only.csv'), index = False)

In [ ]:
meta_df.rename(columns = {name:'__' + name for name in meta_df.columns if name != 'image_id'}, inplace = True)
merged = pd.merge(
    df,
    meta_df,
    how="inner",
    on='image_id',
    left_on=None,
    right_on=None,
    left_index=False,
    right_index=False,
    sort=False,
    suffixes=("??", "__"),
    copy=True,
    indicator=False,
    validate='1:1',
)
merged

In [ ]:
merged.at[0, '__VOILUTSequence'][1].get('LUTData', None)

In [ ]:
merged.at[0, '__VOILUTSequence'][1].get('LUTData', None)

In [ ]:
merged.at[0, '__VOILUTSequence'][0].get('LUTData', None)

In [ ]:
merged.at[0, '__VOILUTSequence'][2]

In [ ]:
merged.columns

In [ ]:
merged.split.value_counts()

In [ ]:
merged['__hasVOILUTSequence'] = merged['__VOILUTSequence'].isna()
merged['__hasLUTDescriptor'] = merged['__LUTDescriptor'].isna()
merged['__hasLUTData'] = merged['__LUTData'].isna()

In [ ]:
merged.rename(columns = {'__PatientAge': 'age'}, inplace = True)

In [ ]:
merged['__PhotometricInterpretation'].value_counts()

In [ ]:
merged['__LUTExplanation'].value_counts()

In [ ]:
merged = merged[['patient_id', 'series_id', 'image_id', 'laterality', 'view', 'age',
       'BIRADS', 'density', 'split', '__SmallestImagePixelValue',
       '__PixelRepresentation', '__SOPInstanceUID', '__PatientWeight', '__WindowWidth',
       '__PixelPaddingValue', '__BodyPartExamined',
       '__StudyInstanceUID', '__ViewPosition', '__LargestImagePixelValue',
       '__PixelSpacing', '__PixelPaddingRangeLimit',
       '__SeriesInstanceUID', '__SamplesPerPixel', '__PresentationLUTShape',
                 '__WindowCenterWidthExplanation', '__WindowCenter',
       '__ImagerPixelSpacing', '__VOILUTFunction', '__RescaleIntercept', '__RescaleSlope',
        '__PhotometricInterpretation', '__Rows', '__Columns',
                 '__hasVOILUTSequence', '__hasLUTDescriptor', '__hasLUTData' ]]
merged

In [ ]:
merged.BIRADS.value_counts()

In [ ]:
merged['cancer'] = merged.BIRADS.apply(lambda x: 1 if x == 5 else 0)

In [ ]:
merged.cancer.value_counts()

In [ ]:
merged.to_csv(os.path.join(ROOT_DIR, 'all_labels.csv'), index = False)

In [ ]:
import dicomsdl

In [ ]:
merged[merged['__hasVOILUTSequence'] == True]

In [ ]:
dcm = dicomsdl.open('/home/dangnh36/datasets/.comp/rsna/external/vindr/images/b8d273e8601f348d3664778dae0e7e0b/d8125545210c08e1b1793a5af6458ee2.dicom')

In [ ]:
dcm.VOILUTSequence[0]

### MiniDDSM

In [ ]:
ROOT_DIR = '/home/dangnh36/datasets/.comp/rsna/external/miniddsm/'
DCM_ROOT_DIR = os.path.join(ROOT_DIR, 'MINI-DDSM-Complete-PNG-16')

In [ ]:
df = pd.read_excel(os.path.join(ROOT_DIR, 'Data-MoreThanTwoMasks', 'Data-MoreThanTwoMasks.xlsx'))
df

In [ ]:
df['patient_id'] = df['fileName'].apply(lambda x: x.split('.')[0])
df['image_id'] = df['patient_id'] + '@' + df['Side'] + '@' + df['View']
df

In [ ]:
df.columns

In [ ]:
df.patient_id.nunique()

In [ ]:
df.image_id.nunique()

In [ ]:
for patient_id, sub_df in tqdm(df.groupby('patient_id')):
    assert sub_df.Status.nunique() == 1

In [ ]:
for patient_id, sub_df in tqdm(df.groupby(['patient_id'])):
    sub_df = sub_df.reset_index(drop = True)
    assert sub_df.Status.nunique() == 1
    status = sub_df.at[0, 'Status']
    no_annotation = True
    for j in range(len(sub_df)):
        if type(sub_df.at[j, 'Tumour_Contour']) == str:
            if len(sub_df.at[j, 'Tumour_Contour']) > 5:
                no_annotation = False
    if status == 'Normal':
        if not no_annotation:
            display(sub_df)
            break
    else:
        if no_annotation:
            display(sub_df)
            break

In [ ]:
cancer_dict = {}
for patient_id, sub_df in tqdm(df.groupby(['patient_id', 'Side'])):
    breast = '@'.join(patient_id)
    sub_df = sub_df.reset_index(drop = True)
    assert sub_df.Status.nunique() == 1
    status = sub_df.at[0, 'Status']
    no_annotation = True
    for j in range(len(sub_df)):
        if type(sub_df.at[j, 'Tumour_Contour']) == str:
            if len(sub_df.at[j, 'Tumour_Contour']) > 5:
                no_annotation = False
    if status == 'Normal':
        if not no_annotation:
            raise AssertionError()
        else:
            cancer_dict[breast] = 0
    elif status == 'Benign':
        cancer_dict[breast] = 0
    elif status == 'Cancer':
        if no_annotation:
            cancer_dict[breast] = 0
        else:
            cancer_dict[breast] = 1
    else:
        raise AssertionError()

In [ ]:
cancer_dict

In [ ]:
df['cancer'] = -1

In [ ]:
for i in tqdm(range(len(df))):
    breast = df.at[i, 'patient_id'] + '@' + df.at[i, 'Side']
    df.at[i, 'cancer'] = cancer_dict[breast]
df

In [ ]:
df[df.cancer == 1].Status.value_counts()

In [ ]:
df[df.cancer == 1]['Tumour_Contour'].isna().any()

In [ ]:
(df[(df.cancer == 0) & (df.Status=='Cancer')]['Tumour_Contour'].apply(lambda x: len(x)) > 1).any()

In [ ]:
df.cancer.value_counts()

In [ ]:
df.columns

In [ ]:
src_dir = os.path.join(ROOT_DIR, 'MINI-DDSM-Complete-PNG-16')
dst_dir = os.path.join(ROOT_DIR, 'png16_v2')
for i in tqdm(range(len(df))):
    status = df.at[i, 'Status']
    patient_id = df.at[i, 'patient_id']
    patient_idx = patient_id.split('_')[1]
    image_id = df.at[i, 'image_id']
    name = df.at[i, 'fileName']
    src_path = os.path.join(src_dir, status, patient_idx, name)
    dst_path = os.path.join(dst_dir, patient_id, f'{image_id}.png')
    os.makedirs(os.path.dirname(dst_path), exist_ok=True)
    shutil.copy2(src_path, dst_path)

In [ ]:
df.Density.value_counts()

In [ ]:
# LEFT, RIGHT --> L, R
df['Side'] = df['Side'].apply(lambda x: x[0])

density_map = {1: 'A', 2: 'B', 3: 'C', 4: 'D', 0: 'A'}
df['Density'] = df['Density'].apply(lambda x: density_map[x])

In [ ]:
df.drop(columns = ['fullPath', 'fileName', 'Tumour_Contour', 'Tumour_Contour2',
                   'Tumour_Contour3', 'Tumour_Contour4', 'Tumour_Contour5',
                   'Tumour_Contour6'], inplace = True)
df.rename(columns = {'View': 'view', 'Side': 'laterality',
                    'Status': 'ddsm_ori_status', 'Age': 'age', 'Density': 'density',
                    }, inplace = True)

In [ ]:
df = df[['patient_id', 'image_id', 'view', 'laterality', 'density', 'age', 'ddsm_ori_status', 'cancer']]
df

In [ ]:
df.to_csv(os.path.join(ROOT_DIR, 'label_v2.csv'), index = False)

### CMMD

In [9]:
ROOT_DIR = '/home/dangnh36/datasets/.comp/rsna/external/cmmd'
DCM_ROOT_DIR = os.path.join(ROOT_DIR, 'CMMD')

In [10]:
df = pd.read_csv(os.path.join(ROOT_DIR, 'CMMD_label.csv'))
df

,ID1,LeftRight,Age,number,abnormality,classification,subtype
0,D1-0001,R,44,2,calcification,Benign,NaN
1,D1-0002,L,40,2,calcification,Benign,NaN
...,...,...,...,...,...,...,...
1870,D2-0748,R,29,2,mass,Malignant,Luminal B
1871,D2-0749,L,42,2,mass,Malignant,Luminal B


In [ ]:
src_dir = DCM_ROOT_DIR
dst_dir = os.path.join(ROOT_DIR, 'dicoms_v2')

for patient_id, sub_df in tqdm(df.groupby('ID1')):
    tmp_dir = os.path.join(src_dir, patient_id)
    tmp2_dirs = os.listdir(tmp_dir)
    assert len(tmp2_dirs) == 1
    tmp3_dirs = os.listdir(os.path.join(tmp_dir, tmp2_dirs[0]))
    assert len(tmp3_dirs) == 1
    src_img_dir = os.path.join(tmp_dir, tmp2_dirs[0], tmp3_dirs[0])
    img_names = os.listdir(src_img_dir)
#     if len(img_names) != sub_df.number.sum():
#         print(patient_id, img_names)
#         print(sub_df.number)
    for img_name in img_names:
        src_img_path = os.path.join(src_img_dir, img_name)
        dst_img_path = os.path.join(dst_dir, patient_id, f'{patient_id}_{img_name}')
        os.makedirs(os.path.dirname(dst_img_path), exist_ok=True)
        shutil.copy2(src_img_path, dst_img_path)
    

In [ ]:
all_patient_ids = []
all_image_ids = []

patients = os.listdir(dst_dir)
for patient in patients:
    patient_dir = os.path.join(dst_dir, patient)
    image_names = os.listdir(patient_dir)
    for image_name in image_names:
        all_patient_ids.append(patient)
        all_image_ids.append(image_name.split('.dcm')[0])
dicom_df = pd.DataFrame({
    'patient_id': all_patient_ids,
    'image_id': all_image_ids
})
dicom_df

In [ ]:
dicom_df = get_dicom_meta(dst_dir, dicom_df, extension = 'dcm')
dicom_df

In [ ]:
df.rename(columns = {'ID1': 'patient_id', 'LeftRight': 'laterality'}, inplace = True)

In [ ]:
dicom_df.rename(columns = {'ImageLaterality': 'laterality'}, inplace = True)

In [ ]:
dicom_df.rename(columns = {name:'__' + name for name in dicom_df.columns if name not in ['patient_id', 'laterality']}, inplace = True)

In [ ]:
dicom_df

In [ ]:
dicom_df.groupby(['patient_id', 'laterality']).count()

In [ ]:
dicom_df.patient_id.nunique()

In [ ]:
df.patient_id.nunique()

In [ ]:
df

In [ ]:
merged = pd.merge(
    dicom_df,
    df,
    how="outer",
    on=['patient_id', 'laterality'],
    left_on=None,
    right_on=None,
    left_index=False,
    right_index=False,
    sort=False,
    suffixes=("__", ""),
    copy=True,
    indicator=False,
#     validate='1:1',
)
merged

In [ ]:
merged.at[0, '__ViewCodeSequence']

In [ ]:
merged.columns

In [ ]:
merged = merged[
    ['patient_id', '__image_id', 'laterality', 'Age', 'number', 'abnormality', 'classification', 'subtype',
     '__VOILUTFunction', '__WindowCenter', '__WindowWidth', '__WindowCenterWidthExplanation',
     '__RescaleSlope', '__RescaleIntercept', '__Columns', '__Rows',
       '__SOPInstanceUID', '__LossyImageCompression',
       '__CodeMeaning', '__StudyID', '__PatientOrientation',
       '__SpecificCharacterSet', '__InstanceCreatorUID', '__PositionerType',
       '__PhotometricInterpretation', '__BitsAllocated',
       '__StudyDate', '__PatientName', '__ContentTime',
       '__PixelIntensityRelationship', '__Modality',
       '__PresentationIntentType', '__AcquisitionTime', '__BodyPartExamined', '__HighBit',
       '__PatientIdentityRemoved', '__BitsStored', '__InstanceCreationTime',
       '__StudyTime', '__SeriesTime', '__PatientBirthDate', '__ImageType',
       '__RescaleType', '__SeriesInstanceUID', '__ReferringPhysicianName',
       '__ContentDate', '__InstanceCreationDate',
       '__PixelIntensityRelationshipSign', '__SeriesNumber',
       '__StudyInstanceUID', '__PatientID', '__DetectorType',
       '__SamplesPerPixel', '__SOPClassUID', '__PixelRepresentation', '__CodeValue',
       '__OrganExposed', '__InstanceNumber', '__AccessionNumber',
       '__SeriesDate', '__AcquisitionDate',
       '__PatientAge',
       '__BurnedInAnnotation', '__PresentationLUTShape',
       '__Manufacturer', '__DeidentificationMethod', '__ImagerPixelSpacing',
       '__PatientSex', ]
]
merged

In [ ]:
merged[merged['classification'].isna()]

In [ ]:
merged.classification.fillna('Normal', inplace = True)

In [ ]:
merged[merged['classification'].isna()]

In [ ]:
merged.columns

In [ ]:
merged.classification.hist()

In [ ]:
merged['cancer'] = merged.classification.apply(lambda x: 1 if x=='Malignant' else 0)

In [ ]:
merged.columns

In [ ]:
merged = merged[['patient_id', '__image_id', 'laterality', 'Age', 'cancer', 'number',
       'abnormality', 'classification', 'subtype', '__VOILUTFunction',
       '__WindowCenter', '__WindowWidth', '__WindowCenterWidthExplanation',
       '__RescaleSlope', '__RescaleIntercept', '__Columns', '__Rows',
       '__SOPInstanceUID', '__LossyImageCompression', '__CodeMeaning',
       '__StudyID', '__PatientOrientation', '__SpecificCharacterSet',
       '__InstanceCreatorUID', '__PositionerType',
       '__PhotometricInterpretation', '__BitsAllocated', '__StudyDate',
       '__PatientName', '__ContentTime', '__PixelIntensityRelationship',
       '__Modality', '__PresentationIntentType', '__AcquisitionTime',
       '__BodyPartExamined', '__HighBit', '__PatientIdentityRemoved',
       '__BitsStored', '__InstanceCreationTime', '__StudyTime', '__SeriesTime',
       '__PatientBirthDate', '__ImageType', '__RescaleType',
       '__SeriesInstanceUID', '__ReferringPhysicianName', '__ContentDate',
       '__InstanceCreationDate', '__PixelIntensityRelationshipSign',
       '__SeriesNumber', '__StudyInstanceUID', '__PatientID', '__DetectorType',
       '__SamplesPerPixel', '__SOPClassUID', '__PixelRepresentation',
       '__CodeValue', '__OrganExposed', '__InstanceNumber',
       '__AccessionNumber', '__SeriesDate', '__AcquisitionDate',
       '__PatientAge', '__BurnedInAnnotation', '__PresentationLUTShape',
       '__Manufacturer', '__DeidentificationMethod', '__ImagerPixelSpacing',
       '__PatientSex']]

merged.rename(columns = {
    '__image_id': 'image_id',
    'Age': 'age',
    'number': '_num_images',
}, inplace = True)
merged

In [ ]:
merged.cancer.hist()

In [ ]:
merged.to_csv(os.path.join(ROOT_DIR, 'label_v2.csv'), index = False)

In [ ]:
merged

In [ ]:
# for _name, sub_df in merged.groupby(['patient_id']):
#     assert sub_df.cancer.nunique() == 1

for _name, sub_df in merged.groupby(['patient_id', 'laterality']):
    assert sub_df.cancer.nunique() == 1

### BMCD

In [ ]:
ROOT_DIR = '/home/dangnh36/datasets/.comp/rsna/external/bmcd/'
DCM_ROOT_DIR = os.path.join(ROOT_DIR, 'dicoms')

In [ ]:
df = pd.read_csv(os.path.join(ROOT_DIR, 'label.csv'))
df.biopsy_result.fillna('NAN', inplace = True)
df

In [ ]:
df.biopsy_result.value_counts()

In [ ]:
df = df[df.biopsy_result.isin(['NAN', 'BENIGN', 'DCIS', 'MALIGNANT'])].reset_index(drop = True)
df

In [ ]:
src_root_dir = DCM_ROOT_DIR
dst_root_dir = os.path.join(ROOT_DIR, 'dicoms_v2')

df['view'] = None
df['image_id'] = None
all_series = []
for i in tqdm(range(len(df))):
    dir_type = df.at[i, 'folder_type']
    dir_name = df.at[i, 'dir']
    dir_parent = 'Suspicious_cases'
    if dir_type == 'normal':
        dir_parent = 'Normal_cases'
    
    src_dir = os.path.join(src_root_dir, str(dir_parent), str(dir_name))
    dst_dir = os.path.join(dst_root_dir, dir_type + '_' + str(dir_name))
    os.makedirs(dst_dir, exist_ok=True)
    names = os.listdir(src_dir)
#     for name in names:
    s = df.loc[i]
    names = [name for name in names if '.dcm' in name.lower()]
    priors = [name for name in names if 'prior' in name]
    recents = [name for name in names if 'recent' in name]
    assert len(priors) == 2 or len(priors) == 0, names
    assert len(recents) == 2, f'{names} and {dir_parent} -{dir_name}'
    for ori_name in names:
        name = ori_name.split('.')[0]
        new_s = s.copy()
        if new_s['biopsy_result'] == 'DCIS' or new_s['biopsy_result'] == 'MALIGNANT':
            if 'prior' in name:
                new_s['biopsy_result'] = 'BENIGN/NORMAL'
        new_name = str(new_s['folder_type']) + '_' + str(new_s['dir']) + name
        new_s['image_id'] =  new_name
        src_path = os.path.join(src_dir, ori_name)
        dst_path = os.path.join(dst_dir, new_name + '.dcm')
        shutil.copy2(src_path, dst_path)
        
        new_s['view'] = name.split('_')[0]
        all_series.append(new_s)

In [ ]:
df = pd.DataFrame(all_series)
df

In [ ]:
df.biopsy_result.value_counts()

In [ ]:
print(df[df.biopsy_result.isin(['MALIGNANT', 'DCIS'])]['image_id'].tolist())

In [ ]:
df.columns

In [ ]:
df['cancer'] = df.biopsy_result.apply(lambda x: 1 if x in ['MALIGNANT', 'DCIS'] else 0)
df.cancer.value_counts()

In [ ]:
df.rename(columns = {
    'dir': 'patient_id',
}, inplace = True)
df

In [ ]:
df['patient_id'] = df['folder_type'] + '_' +  df['patient_id'].apply(str)

In [ ]:
dicom_df = get_dicom_meta(dst_root_dir, df, extension = 'dcm')

In [ ]:
dicom_df.dropna(axis=1, how='all', inplace = True)

In [ ]:
dicom_df

In [ ]:
dicom_df.rename(columns = {name:'__' + name for name in dicom_df.columns if name != 'image_id'}, inplace = True)
merged = pd.merge(
    df,
    dicom_df,
    how="inner",
    on='image_id',
    left_on=None,
    right_on=None,
    left_index=False,
    right_index=False,
    sort=False,
    suffixes=("", "__"),
    copy=True,
    indicator=False,
    validate='1:1',
)
merged

In [ ]:
merged

In [ ]:
print(list(merged.columns))

In [ ]:
merged = merged[['patient_id', 'laterality', 'age', 'density', 'BIRADS', 'biopsy_result', 'folder_type', 'view', 'image_id', 'cancer', '__WindowWidth', '__WindowCenter', '__WindowCenterWidthExplanation', '__RescaleType', '__RescaleSlope', '__RescaleIntercept', '__Rows', '__Columns', '__BitsAllocated', '__BitsStored']]
merged

In [ ]:
merged.to_csv(os.path.join(ROOT_DIR, 'label_v2.csv'), index = False)

### CDD-CESM

In [5]:
ROOT_DIR = '/home/dangnh36/datasets/.comp/rsna/external/cdd_cesm/'
DCM_ROOT_DIR = os.path.join(ROOT_DIR, 'low_energy')

In [ ]:
df = pd.read_csv(os.path.join(ROOT_DIR, 'cdd_cesm.csv'))


In [ ]:
df.Type.value_counts()

In [ ]:
df['image_id'] = df['Image_name'].apply(lambda x: x.replace('DM_', '').replace('CM_', ''))
# df['patient_id'] = df['Image_name'].apply(lambda x: x.split('_')[0])
df

In [ ]:
df['Pathology Classification/ Follow up'].value_counts()

In [ ]:
for image_name, sub_df in df.groupby('image_id'):
    sub_df = sub_df.reset_index(drop = True)
    if sub_df.at[0, 'Pathology Classification/ Follow up'] == 'Malignant':
        assert sub_df['Pathology Classification/ Follow up'].nunique() == 1, sub_df

In [ ]:
df = df[df.Type == 'DM'].reset_index(drop = True)
df

In [ ]:
df.columns

In [ ]:
df.rename(columns = {
    'Image_name': 'image_name',
    'Patient_ID': 'patient_id',
    'Side': 'laterality',
    'Age': 'age',
    'Breast density (ACR)': 'density',
    'Findings': 'findings',
    'View': 'view',
    'Tags': 'tags',
    'Machine': 'machine_id',
    'Pathology Classification/ Follow up': 'classification',
}, inplace = True)
df.columns

In [ ]:
df = df[['patient_id', 'image_id', 'laterality', 'view', 'image_name', 'age', 'density', 'BIRADS',
       'findings', 'tags', 'machine_id', 'classification', 'Type']]
df

In [ ]:
df.classification.value_counts()

In [ ]:
df['cancer'] = df.classification.apply(lambda x: 1 if x == 'Malignant' else 0)
df

In [ ]:
df.image_id.nunique()

In [ ]:
df.cancer.value_counts()

In [8]:
df.to_csv(os.path.join(ROOT_DIR, 'label_v2.csv'), index = False)

In [6]:
df = pd.read_csv(os.path.join(ROOT_DIR, 'label_v2.csv'))
df

,patient_id,image_id,laterality,view,image_name,age,density,BIRADS,findings,tags,machine_id,classification,Type,cancer
0,1,P1_L_MLO,L,MLO,P1_L_DM_MLO,46,C,4,Architectural distortion,"suspicious, distortion",1,Malignant,DM,1
1,2,P2_R_CC,R,CC,P2_R_DM_CC,31,C,3$2,Multiple equal density masses$Benign macrocal...,"benign, masses, macrocalcification",2,Benign,DM,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1001,326,P326_L_CC,L,CC,P326_L_DM_CC,51,C,3,Asymmetrical increased density,asymmetry,1,Malignant,DM,1
1002,326,P326_L_MLO,L,MLO,P326_L_DM_MLO,51,C,3,Asymmetrical increased density,asymmetry,1,Malignant,DM,1


In [7]:
df['image_id']  = df['image_id'].apply(lambda x: x.strip())